# Aarhus Workforce Forecast 2026-2030
# Based on historical data with scenario modeling for spouse employment impact

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from datetime import datetime
import sys



In [2]:
# Set plotting style
# Import custom styling from parent directory
sys.path.append('..')
from custom_viz_styling import setup_styling

# Apply custom visualization styling
setup_styling()

Custom visualization styling applied successfully.


In [3]:
# Warning about forecasting limitations
print("⚠️ FORECASTING LIMITATION WARNING ⚠️")
print("This notebook projects future trends based solely on historical patterns.")
print("Real-world events (economic shifts, policy changes, geopolitical events)")
print("will likely cause significant deviations from these projections.")
print("Use these forecasts as a baseline, not as definitive predictions.\n")

⚠️ FORECASTING LIMITATION WARNING ⚠️
This notebook projects future trends based solely on historical patterns.
Real-world events (economic shifts, policy changes, geopolitical events)
will likely cause significant deviations from these projections.
Use these forecasts as a baseline, not as definitive predictions.



In [4]:
# Load the datasets
def load_data():
    """Load and prepare all datasets"""
    # Load work permits data
    work_permits = pd.read_csv('../data/processed/fact_work_permits.csv')
    work_permits['Total workers'] = work_permits['Full time third country workers'] + work_permits['Full time EU workers']
    
    # Load migration data
    migration = pd.read_csv('../data/processed/fact_migration.csv')
    migration['Net migration'] = migration['Immigration foreign citizens 20-64 years'] - migration['Emigration foreign citizens 20-64 years']
    
    # Load population data
    population = pd.read_csv('../data/processed/fact_population.csv')
    population.rename(columns={' "Foreign citizens 20-64 years"': 'Foreign citizens 20-64 years'}, inplace=True)
    
    # Load retention percentage data
    retention = pd.read_csv('../data/processed/fact_retention_percentage.csv')
    
    # Load all permits data
    all_permits = pd.read_csv('../data/processed/fact_all_permits.csv')
    
    return work_permits, migration, population, retention, all_permits

work_permits, migration, population, retention, all_permits = load_data()

In [5]:
# Display summary of data
print("Data Summary (2021-2025):")
print(f"Foreign worker population (2025): {population['Foreign citizens 20-64 years'].iloc[-1]:,}")
print(f"Latest annual net migration (2024): {migration['Net migration'].iloc[-1]:,}")
print(f"Total international workers (2025): {work_permits['Total workers'].iloc[-1]:,}")
print("---")

Data Summary (2021-2025):
Foreign worker population (2025): 32,479
Latest annual net migration (2024): 1,459
Total international workers (2025): 6,345
---


In [6]:
def generate_forecast_table(historical_dfs, target_columns, years_to_forecast=5, include_cv=True, include_r2=True):
    """
    Generate a forecast table with predictions and realistic uncertainty metrics
    
    Parameters:
    -----------
    historical_dfs : dict
        Dictionary of {name: dataframe} containing historical data
        Each dataframe must have a 'Year' column
    target_columns : dict of lists
        Dictionary of {name: [column_names]} specifying which columns to forecast in each dataframe
    years_to_forecast : int, default=5
        Number of years to forecast
    include_cv : bool, default=True
        Whether to include coefficient of variation in output
    include_r2 : bool, default=True
        Whether to include R-squared in output
    
    Returns:
    --------
    forecast_table : pandas.DataFrame
        A table with columns: Year, Category, Metric, Forecast, Uncertainty
    """
    import pandas as pd
    import numpy as np
    from sklearn.linear_model import LinearRegression
    
    # Get last year from data
    last_year = max([df['Year'].max() for df in historical_dfs.values()])
    
    # Generate forecast years
    forecast_years = range(last_year + 1, last_year + years_to_forecast + 1)
    
    # Initialize results table
    results = []
    
    # Process each dataset
    for category, df in historical_dfs.items():
        columns = target_columns[category]
        
        for column in columns:
            # Skip columns with missing data
            if df[column].isnull().any():
                continue
                
            # Create model
            X = df['Year'].values.reshape(-1, 1)
            y = df[column].values
            
            model = LinearRegression()
            model.fit(X, y)
            
            # Get y-values for calculation
            y_pred = model.predict(X)
            absolute_deviations = np.abs(y - y_pred)
            
            # Calculate Mean Absolute Error (MAE)
            mae = np.mean(absolute_deviations)
            
            # Coefficient of variation (%)
            mean_value = np.mean(y)
            cv = (np.std(y) / mean_value) * 100 if mean_value != 0 else np.inf
            
            # Get R-squared
            r2 = model.score(X, y)
            
            # Calculate Mean Absolute Percentage Error (MAPE)
            # Avoid division by zero
            valid_y = y[y != 0]
            valid_pred = y_pred[y != 0]
            if len(valid_y) > 0:
                mape = np.mean(np.abs((valid_y - valid_pred) / valid_y)) * 100
            else:
                mape = np.nan
            
            # Generate forecast for future years
            for i, year in enumerate(forecast_years):
                # Calculate how many years into the future
                years_out = i + 1
                
                # Get forecast value
                forecast_value = model.predict([[year]])[0]
                
                # Calculate uncertainty based on MAE with increasing factor
                # We'll use a square root scaling to reduce the growth rate
                # This provides more realistic uncertainty that doesn't grow too quickly
                uncertainty_factor = np.sqrt(years_out)
                uncertainty = mae * uncertainty_factor
                
                # Cap uncertainty as percentage of forecast value
                max_uncertainty_pct = 50  # Maximum 50% uncertainty
                max_uncertainty = abs(forecast_value) * (max_uncertainty_pct / 100)
                uncertainty = min(uncertainty, max_uncertainty)
                
                # For values near zero, use a minimum absolute uncertainty
                min_uncertainty = 50  # Minimum uncertainty of 50 units
                if abs(forecast_value) < 1000:
                    uncertainty = max(uncertainty, min_uncertainty)
                
                # Create a row for the results table
                result_row = {
                    'Year': year,
                    'Category': category,
                    'Metric': column,
                    'Forecast': round(forecast_value),
                    'Uncertainty (±)': round(uncertainty)
                }
                
                # Add optional columns
                if include_cv:
                    result_row['CV (%)'] = round(cv, 1)
                if include_r2:
                    result_row['R²'] = round(r2, 3)
                
                # Add MAPE
                result_row['MAPE (%)'] = round(mape, 1) if not np.isnan(mape) else None
                
                # Add to results
                results.append(result_row)
    
    # Convert to DataFrame and sort
    forecast_table = pd.DataFrame(results)
    forecast_table = forecast_table.sort_values(['Year', 'Category', 'Metric'])
    
    return forecast_table

# Example usage:

# Define all the granular columns to forecast
columns_to_forecast = {
    'Foreign Workers': [
        'Full time third country workers', 
        'Full time EU workers',
        'Total workers'
    ],
    'Migration': [
        'Immigration foreign citizens 20-64 years',
        'Emigration foreign citizens 20-64 years',
        'Net migration'
    ],
    'Foreign Population': [
        'Foreign citizens 20-64 years'
    ],
    'Work Permits': [
        'Work permit 3rd country',
        'Work permit EU',
        'Study permit',
        'Family reunification',
        'Permanent residency',
        'Ukraine Emergency Law'
    ]
}

# Create the forecast table with all granular metrics
detailed_forecast = generate_forecast_table(
    historical_dfs={
        'Foreign Workers': work_permits,
        'Migration': migration,
        'Foreign Population': population,
        'Work Permits': all_permits
    },
    target_columns=columns_to_forecast,
    years_to_forecast=5
)

# Display the table
print(detailed_forecast)

# Save to CSV
detailed_forecast.to_csv('detailed_forecast_table_2026_2030.csv', index=False)

# Create a pivot table grouped by year and category
pivot_by_category = pd.pivot_table(
    detailed_forecast,
    values=['Forecast', 'Uncertainty (±)'],
    index=['Year', 'Category', 'Metric'],
    aggfunc='first'
).reset_index()

print("\nPivot table by category:")
print(pivot_by_category)


# Function to assess forecast reliability
def assess_forecast_reliability(forecast_table):
    """
    Assess and categorize the reliability of each forecast
    
    Parameters:
    -----------
    forecast_table : pandas.DataFrame
        The forecast table generated by generate_forecast_table
        
    Returns:
    --------
    reliability_table : pandas.DataFrame
        The forecast table with added reliability assessment
    """
    import pandas as pd
    import numpy as np
    
    # Create a copy to avoid modifying the original
    reliability_table = forecast_table.copy()
    
    # Calculate uncertainty as percentage of forecast
    reliability_table['Uncertainty (%)'] = (reliability_table['Uncertainty (±)'] / reliability_table['Forecast'].abs()) * 100
    
    # Reliability categories based on uncertainty percentage and R²
    conditions = [
        (reliability_table['Uncertainty (%)'] <= 10) & (reliability_table['R²'] >= 0.9),
        (reliability_table['Uncertainty (%)'] <= 25) & (reliability_table['R²'] >= 0.7),
        (reliability_table['Uncertainty (%)'] <= 40) & (reliability_table['R²'] >= 0.5),
        (reliability_table['Uncertainty (%)'] <= 60),
        (reliability_table['Uncertainty (%)'] > 60)
    ]
    
    categories = [
        'High Reliability',
        'Good Reliability',
        'Moderate Reliability',
        'Low Reliability',
        'Very Low Reliability'
    ]
    
    reliability_table['Reliability'] = np.select(conditions, categories, default='Unknown')
    
    # Round the uncertainty percentage
    reliability_table['Uncertainty (%)'] = reliability_table['Uncertainty (%)'].round(1)
    
    return reliability_table

# Example for reliability assessment:

# Assess the reliability of forecasts
reliability_assessment = assess_forecast_reliability(detailed_forecast)

# Print summary of reliability by metric
reliability_summary = reliability_assessment.groupby(['Category', 'Metric', 'Reliability']).size().reset_index(name='Count')
print("\nReliability Assessment Summary:")
print(reliability_summary)

# Save the reliability assessment
reliability_assessment.to_csv('forecast_reliability_assessment.csv', index=False)

# Display metrics with highest and lowest reliability
print("\nMost Reliable Metrics:")
most_reliable = reliability_assessment[reliability_assessment['Year'] == 2026].sort_values(['Uncertainty (%)'])
print(most_reliable[['Category', 'Metric', 'Forecast', 'Uncertainty (%)', 'Reliability']].head(5))

print("\nLeast Reliable Metrics:")
least_reliable = reliability_assessment[reliability_assessment['Year'] == 2026].sort_values(['Uncertainty (%)'], ascending=False)
print(least_reliable[['Category', 'Metric', 'Forecast', 'Uncertainty (%)', 'Reliability']].head(5))


    Year            Category                                   Metric  \
30  2026  Foreign Population             Foreign citizens 20-64 years   
5   2026     Foreign Workers                     Full time EU workers   
0   2026     Foreign Workers          Full time third country workers   
10  2026     Foreign Workers                            Total workers   
20  2026           Migration  Emigration foreign citizens 20-64 years   
..   ...                 ...                                      ...   
59  2030        Work Permits                      Permanent residency   
49  2030        Work Permits                             Study permit   
64  2030        Work Permits                    Ukraine Emergency Law   
39  2030        Work Permits                  Work permit 3rd country   
44  2030        Work Permits                           Work permit EU   

    Forecast  Uncertainty (±)  CV (%)     R²  MAPE (%)  
30     34485              343     8.6  0.970       1.2  
5       5